# Factor Selection
## Growth Factor

In [125]:
from WindPy import w
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.api import het_white, het_breuschpagan
from statsmodels.stats.sandwich_covariance import cov_hac
from statsmodels.tsa.seasonal import STL

In [2]:
w.start()

Wind.Cosmos.Base V1.7 compiled time is Apr 17 2023, BuildType:Release, CPUArch:X64, GCC Version:Apple LLVM 13.0.0 (clang-1300.0.29.30)
Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2021 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [82]:
start_date = "2008-12-01" # actually from 2010, extract data for preprocessing
end_date = "2024-05-31"

In [20]:
# takes a dict of indicators and their corresponding ticker in Wind to
# create a DataFrame containing the data extracted from Wind API
def extract_inds(inds):
    df = pd.DataFrame()
    for ind in inds:
        raw = w.edb(inds[ind], start_date, end_date)
        # Extract data from the Wind API response
        times = raw.Times
        data = raw.Data[0]
        # Convert times to pandas datetime format
        times = pd.to_datetime([str(time) for time in times], format="%Y-%m-%d")
        
        # Create a temporary DataFrame for the current indicator
        temp_df = pd.DataFrame(data, index=times, columns=[ind])
        
        # Combine the temporary DataFrame with the main DataFrame
        if df.empty:
            df = temp_df
        else:
            df = df.join(temp_df, how='outer')
    return df

In [28]:
def lag_data(df, periods=1):
    df_lagged = df.shift(periods)
    df_lagged = df_lagged.iloc[periods:]  # Remove the first rows
    return df_lagged

In [6]:
# STL seasonality processing
def apply_stl(df, columns, seasonal=13):
    df_adjusted = df.copy()
    for column in columns:
        stl = STL(df[column], seasonal=seasonal)
        result = stl.fit()
        df_adjusted[column] = result.seasonal  # Replace with seasonally adjusted component
    return df_adjusted

In [7]:
# YoY Difference Processing
def to_yoy(df, columns):
    df_yoy = df.copy()
    for column in columns:
        df_yoy[column] = df[column] - df[column].shift(12)
    df_yoy = df_yoy.iloc[12:]  # Remove the first 12 rows
    return df_yoy

In [109]:
# Get USD/CNH Exchange Rate
fx_data = w.edb("M0000185", "2009-01-01", end_date, Period="M")
# Parse the data
times = fx_data.Times
data = fx_data.Data[0]
times = pd.to_datetime([str(time) for time in times], format="%Y-%m-%d")

# Construct the DataFrame
df_forex = pd.DataFrame(data, index=times, columns=["USD/CNY Central Parity Rate"])
df_forex = to_yoy(df_forex, df_forex.columns)

# Display the DataFrame
df_forex

,USD/CNY Central Parity Rate
2010-01-29,-0.0110
2010-02-26,-0.0110
2010-03-31,-0.0096
2010-04-30,0.0013
2010-05-31,-0.0044
...,...
2024-01-31,0.3435
2024-02-29,0.1517
2024-03-29,0.2233
2024-04-30,0.1823


In [139]:
# for factor selection, regress on the exchang rate to observe Beta, T-Value, and R-Squared
# and record the median values after 1000 iterations
def regress(indicator_df, forex_df, target_column='USD/CNY Central Parity Rate', n_iterations=1000, min_period_length=24):
    # Initialize a DataFrame to store regression results
    df_forex.index = indicator_df.index
    results = []

    for i in range(n_iterations):
        # Resample with replacement
        sample_indices = np.random.choice(indicator_df.index, size=len(indicator_df), replace=True)
        sample_growth_df = indicator_df.loc[sample_indices]
        sample_forex_df = forex_df.loc[sample_indices]
        
        # Randomly select start and end dates
        start_idx = np.random.randint(0, len(sample_growth_df) - min_period_length)
        end_idx = start_idx + min_period_length
        
        # Subset the data to ensure period longer than 2 years
        sample_growth_period = sample_growth_df.iloc[start_idx:end_idx]
        sample_forex_period = sample_forex_df.iloc[start_idx:end_idx]
        
        # Perform regression for each indicator
        for indicator in indicator_df.columns:
            X = sample_growth_period[[indicator]]
            X = sm.add_constant(X)
            y = sample_forex_period[target_column]
            
            model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags':1})
            beta = model.params[indicator]
            t_value = model.tvalues[indicator]
            r_squared = model.rsquared
            
            # Append the results
            results.append({
                'Indicator': indicator,
                'Beta': beta,
                'T-Value': t_value,
                'R-Squared(%)': r_squared * 100  # Convert to percentage
            })
            
    # Convert results to DataFrame
    growth_results_df = pd.DataFrame(results)

    # Compute median values
    growth_median_results = growth_results_df.groupby('Indicator').median().reset_index()

    # Format the median results
    growth_median_results['Beta'] = growth_median_results['Beta'].map("{:.2f}".format)
    growth_median_results['T-Value'] = growth_median_results['T-Value'].map("{:.2f}".format)
    growth_median_results['R-Squared(%)'] = growth_median_results['R-Squared(%)'].map("{:.2f}%".format)

    return growth_median_results

In [9]:
# consider key growth indicators for China
china_growth_inds = {
    # production method
    # general macro indicators
    'GDP_China(%)': 'M0039354',
    'Industrial_Growth(%)': 'M0000545',
    'PMI': 'M0017126',
    'PMI_manufacture': 'M0017127',
    'PMI_new_orders': 'M0017128',
    # by industry
    'electricity(%)': 'S0027013',
    'concrete(%)': 'S0027703',
    'steel(%)': 'S0027375',
    'automobile(%)': 'S0027908',
    'railroad_cargo(%)': 'S0036034',
    # expenditure method
    # investments
    'fixed_asset(%)': 'M0000273',
    'real_estate(%)': 'S0029657',
    'infrastructure(%)': 'M5440435',
    'manufacture(%)': 'M0000357',
    # consumption
    'retail(%)': 'M0001428',
    'automobile_sales(%)': 'S6114593',
    'tractor_sales(%)': 'S6002167',
    'commercial_RE_area(%)': 'S0073300',
    'commercial_RE_revenue(%)': 'S0049591',
    # net exports
    'import_export(%)': 'M0000605',
    'export(%)': 'M0000607',
    'import(%)': 'M0000609',
    # income method
    'govt_revenue(%)': 'M0046169',
    'industrials_biz_income(%)': 'M0000555',
    'industrials_tot_profits(%)': 'M0000557'
}
# for standardizing the PMI data
china_nonyoy_cols = ['PMI', 'PMI_manufacture', 'PMI_new_orders']

In [66]:
# actual step retrieving data from Wind API
df_china_growth = extract_inds(china_growth_inds)
df_china_growth = lag_data(df_china_growth)  # lag data by 1 period
df_china_growth = to_yoy(df_china_growth, china_nonyoy_cols)
df_china_growth = df_china_growth.bfill().interpolate(method="linear")  # interpolate missing values
df_china_growth = apply_stl(df_china_growth, df_china_growth.columns[1:], seasonal=13)
df_china_growth.head(10)


,GDP_China(%),Industrial_Growth(%),PMI,PMI_manufacture,PMI_new_orders,electricity(%),concrete(%),steel(%),automobile(%),railroad_cargo(%),...,automobile_sales(%),tractor_sales(%),commercial_RE_area(%),commercial_RE_revenue(%),import_export(%),export(%),import(%),govt_revenue(%),industrials_biz_income(%),industrials_tot_profits(%)
2010-01-31,11.9,-0.096970,2.676930,4.250446,4.295375,0.601227,-3.171591,0.625182,19.132004,1.014852,...,8.980903,7.264556,1.598908,3.587245,-5.436913,-7.867885,-2.066218,-7.074421,1.271387,4.131011
2010-02-28,12.2,2.783520,1.274089,2.060007,1.695931,6.630864,8.592328,5.101400,26.032968,1.212978,...,23.171006,46.177350,4.171527,7.164816,3.980726,-1.650477,14.438124,3.291982,-1.650102,2.322865
2010-03-31,12.2,0.323882,0.148882,0.227521,-0.081883,-1.960675,-5.596200,1.990873,-7.204441,0.227684,...,-4.601357,-19.095841,5.233529,8.981636,-0.492400,0.768446,-3.225484,1.876411,-0.776474,6.137579
2010-04-30,12.2,0.471551,-0.634509,-1.420660,-0.819768,0.754225,1.236998,2.104911,-3.118176,0.805208,...,-0.590846,12.352918,3.482161,4.557782,1.135167,-1.911502,4.450236,2.227174,-0.722639,-5.372916
2010-05-31,10.8,-0.273816,-0.712248,-1.000702,-1.176908,0.779088,0.088304,3.276847,-8.572519,-0.218718,...,-7.026914,0.387010,0.731982,1.376766,-0.202149,0.511089,-1.812951,1.657076,-0.592664,-2.811661
2010-06-30,10.8,-0.407686,-0.606715,-0.730296,-1.275922,0.575272,0.129788,1.702735,-7.269195,-0.427904,...,-6.265093,4.947086,-1.558555,-2.483254,4.114401,5.766158,1.343908,1.082689,-0.014208,-0.174990
2010-07-31,10.8,-0.729037,-0.614818,-0.850970,-1.010721,-0.935082,0.042277,-1.700767,-7.755009,-1.470858,...,-5.998010,-6.973331,-2.541324,-4.412250,-0.999995,2.650109,-5.773855,0.133510,-1.131388,-6.994809
2010-08-31,9.9,-0.719035,-0.419007,-0.808744,-0.629003,-0.843911,0.449640,-1.843696,-7.869350,-2.053152,...,-7.707560,-11.450918,-3.347716,-5.273329,-2.195537,1.002463,-6.217931,-0.417073,-0.480287,-4.060486
2010-09-30,9.9,-0.515715,-0.416540,-0.764131,-0.333367,-0.352534,-1.426724,-3.093636,-4.937814,-1.706694,...,-4.119795,-11.718382,-3.411548,-5.254498,1.174948,2.134809,-0.341625,-0.734189,0.271311,-0.970916
2010-10-31,9.9,-0.310241,-0.045951,-0.080899,0.172197,-1.706833,-0.997469,-2.974243,-2.319353,0.474085,...,-1.914242,-9.962740,-2.042651,-3.079432,-2.870760,-2.210340,-3.801352,-0.909689,0.434366,-0.140897


In [140]:
china_growth_ctrb = regress(df_china_growth, df_forex)
china_growth_ctrb

,Indicator,Beta,T-Value,R-Squared(%)
0,GDP_China(%),-0.05,-3.53,21.87%
1,Industrial_Growth(%),-0.00,-0.08,3.27%
2,PMI,0.00,0.03,1.70%
3,PMI_manufacture,-0.00,-0.01,1.57%
4,PMI_new_orders,0.00,0.04,1.53%
5,automobile(%),0.00,0.00,1.23%
6,automobile_sales(%),-0.00,-0.06,2.89%
7,commercial_RE_area(%),-0.00,-0.03,2.60%
8,commercial_RE_revenue(%),-0.00,-0.01,2.28%
9,concrete(%),-0.00,-0.01,2.17%


In [155]:
us_growth_inds = {
        # production method
        # general macro indicators
        'GDP_US': 'G1112986',
        'industrial production': 'G1109285',
        'PMI': 'G0002323',
        'PMI_output': 'G0008346',
        'PMI_new_order': 'G0008345', 
        # by industry
        'electricity': 'G1130815',
        'concrete_rebar': 'D5715886',
        'crude_steel': 'S5709965',
        'automobile': 'G1121656',
        # investment
        'real_estate_loans': 'T7212500',
        # consumption
        'new_house_sales': 'G0003650',
        'auto_sales': 'G1121638',
        'food_and_retail_sales': 'G1113053',
        'import_export': 'G1113168',
        'export': 'G1113166',
        'import': 'G1100355'
}
us_nonyoy_cols = ['PMI', 'PMI_output', 'PMI_new_order', 'electricity', 'concrete_rebar', 'automobile', 'real_estate_loans', 'new_house_sales', 'auto_sales']

In [156]:
df_us_growth = extract_inds(us_growth_inds)
df_us_growth = lag_data(df_us_growth)  # lag data by 1 period
df_us_growth = to_yoy(df_us_growth, us_nonyoy_cols)
df_us_growth = df_us_growth.bfill().interpolate(method="linear")  # interpolate missing values
df_us_growth = apply_stl(df_us_growth, df_us_growth.columns[1:], seasonal=13)
df_us_growth.head(10)

,GDP_US,industrial production,PMI,PMI_output,PMI_new_order,electricity,concrete_rebar,crude_steel,automobile,real_estate_loans,new_house_sales,auto_sales,food_and_retail_sales,import_export,export,import
2010-01-31,0.1055,-2.104912,-0.109984,3.127746,4.729250,-6662.072806,0.008160,-8.482444,-12.759727,5921.617276,9.797000,5.035175,-0.409410,17.082833,-4.496115,-6.621462
2010-02-28,1.7450,-1.121366,0.035345,1.962071,1.681203,-5526.198138,0.012489,-7.912877,4.240988,5536.514186,26.043872,-4.059319,-1.358080,7.143253,-1.732862,-3.082494
2010-03-31,1.7450,-0.829593,0.852887,0.751445,2.678163,193.406849,0.008910,-5.860049,2.478132,3879.584470,9.042721,21.365944,0.398335,-3.677337,-2.148353,-0.438117
2010-04-30,1.7450,-0.005747,1.658033,2.283236,1.184880,-2261.637462,0.008468,3.019088,1.930253,1625.230618,20.258997,44.896011,1.855596,-4.329208,0.929023,1.191207
2010-05-31,2.9141,0.582113,1.275378,2.931003,0.033230,-2330.742255,0.001408,9.602000,3.240757,-148.636743,27.306847,20.949567,0.894647,-2.598684,2.661588,1.938611
2010-06-30,2.9141,1.115886,0.867969,2.088439,-0.340290,1202.864307,-0.007144,9.576427,5.844819,-2497.781550,3.984533,19.271348,0.392128,-14.095850,2.232160,3.556577
2010-07-31,2.9141,1.205672,0.044030,-2.292315,-0.582984,2369.547850,-0.012743,8.281168,5.581596,-2363.753420,-9.516234,10.349294,-0.685235,-5.172639,1.585113,2.942933
2010-08-31,3.3443,0.755645,-0.860765,-3.189354,-1.754819,8618.050847,-0.012456,0.559381,1.089004,-2182.786231,-23.561963,-33.987991,-0.957430,2.129256,0.173301,0.557158
2010-09-30,3.3443,0.294989,-1.332243,-3.201606,-4.570043,2145.806128,-0.010268,-0.882495,-0.813841,-2374.372482,-25.564591,-82.957142,-0.557575,-4.174901,1.544052,1.489362
2010-10-31,3.3443,-0.014635,-1.113288,-1.482798,-2.631970,1667.603620,-0.001448,-3.942923,-4.842786,-2190.109361,-10.647533,7.574352,0.279172,3.589158,-0.225256,-0.237553


In [157]:
us_growth_ctrb = regress(df_us_growth, df_forex)
us_growth_ctrb

,Indicator,Beta,T-Value,R-Squared(%)
0,GDP_US,-0.03,-1.35,6.00%
1,PMI,-0.02,-0.13,1.52%
2,PMI_new_order,-0.01,-0.15,1.23%
3,PMI_output,-0.01,-0.12,1.62%
4,auto_sales,-0.00,-0.24,1.44%
5,automobile,-0.00,-0.04,1.32%
6,concrete_rebar,-1.73,-0.16,2.10%
7,crude_steel,0.00,0.06,1.32%
8,electricity,0.00,0.10,1.49%
9,export,0.00,0.07,2.12%


It appears that of all the indicators picked intended to represent growth in China and US's economy, most of the indicators have very little contributions to the movement of USD/CNH. China's GDP appears to have a more direct impact, as its T-value indicates significance and has a relatively high R-squared value of roughly 20%, thus we select China's GDP as a component of our growth factor. Meanwhile, the US GDP appears less impactful than its counterpart but is still the most influential indicator amongst the US indicators. Hence, we also select the US GDP as a component of the growth factor. 

Both China and US GDP having slight negative beta on the exchange rate (USD/CNY) can be explained by considering the economic dynamics between the two countries. In China's case, a growing Chinese economy might lead to increased export which prompts RMB to appreciate against USD, while a growing US economy often leads to increased imports, which increases demands for Chinese goods and thereby exerts downward pressure on USD/CNH exchange rate.